## Functions

In [30]:
import LyndonWords
import importlib
import AnalysisFuncs
importlib.reload(LyndonWords)
importlib.reload(AnalysisFuncs)
from AnalysisFuncs import *
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def gen_permutations(type,n):
    for perm in itertools.permutations(np.array(range(n+1))):
        yield rootSystem(perm,type)

### Monotonicity Stuff

In [ ]:
def monotonicity_conj_perms(rootSystems,total=0,k=3):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(monotonicity_conj,k=k),rootSystems),total=total):
            if(not result.truthValue):
                yield result        
#TODO: Updated bases on new claim on monotonicity
def check_monotone_increasing_imp_cond(rootSystemPermutations):
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.SL(rootsys.delta)
        temp = rootsys.check_monotonicity("Increasing")
        monotoneIncreasing = [rootsys.SL(i[0] + rootsys.delta)[0] for i in temp]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [ ]:
def check_cond_imp_monotone_increasing(rootSystemPermutations):
    returnArr = []
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.SL(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.SL(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.get_monotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.SL(base)[0])))
    return returnArr

### Printing and Exporting

In [ ]:
def print_by_length(rootSystem:rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.SL(comb):
                print(" "+str(i))

In [34]:
def print_by_base_word(rootSystem:rootSystem,comb):
    k=0
    is_imaginary = rootSystem.is_imaginary(comb)
    for i in rootSystem.get_chain(comb):
        if(is_imaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not is_imaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [ ]:
def export_by_length(rootsys:rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootsys.ordering)+"\n")
    for k in range(rootsys.k + 1):
        for i in rootsys.baseWeights:
            comb = i + rootsys.delta * k
            for i in rootsys.SL(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

### Delta Pattern Detection

In [36]:
def write_delta_types_to_file(rootSystems,file="out.txt",total=0,notOnlyAllStandard=True):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, bracketing, splitting type, inserted imaginary index, left factor, right factor\n")
    deltaTypesCol:deltaTypesCollection
    first:rootSystem
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        deltaTypesCol = generate_delta_types(first)
        if(notOnlyAllStandard):
            if(deltaTypesCol.not_all_standard()):
                fileobj.write(deltaTypesCol.to_csv())
    with mp.Pool() as p:
        for deltaTypesCol in tqdm(p.imap_unordered(generate_delta_types,rootSystems),total=total):
            if(notOnlyAllStandard):
                if(not deltaTypesCol.not_all_standard()):
                    continue
            fileobj.write(deltaTypesCol.to_csv())
    fileobj.close()

In [37]:
def check_standard_fac_same_parrallel(rootSystems,total=0,k=2):
    results = []
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        result = check_standard_fac_same(first)
        if(result != True):
            yield result
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_standard_fac_same,k=k),rootSystems,chunksize=5),total=total):
            if(result != True):
                yield result
    return results

### Imaginary Words Stuff

In [38]:
def check_h_deltas_repeat(perms):
    excepts = []
    i:rootSystem
    for i in perms:
        hs = [j.hs for j in i.get_chain(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [ ]:
def check_one_smallest_last(perms):
    rootsys:rootSystem
    for rootsys in perms:
        if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
            if(rootsys.SL(rootsys.delta)[0][-1].rootIndex != rootsys.ordering[1].rootIndex):
                yield (str(rootsys.ordering),rootsys.SL(rootsys.delta)[0].noCommas(),rootsys.ordering[1].rootIndex)

### Rootsystem equivalences

In [ ]:
def compare_rootsystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.SL(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.SL(i[j].weights)[j]))

In [41]:
def generate_equivalences(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compare_rootsystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

### Periodicity

In [42]:
def get_max_periodicity(rootSystems,total=0):
    max = 0
    maxOrdering = ""
    maxRoot = []
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(max_periodicity_rootSystem,rootSystems),total=total):
            if(result.periodicity > max):
                max = result.periodicity
                maxOrdering = result.ordering
                maxRoot= result.maxRoot
    return (max,maxOrdering,maxRoot)

In [43]:
def periodicity_conj(rootsys:rootSystem,deltas, weights) -> int:
    if(weights[rootsys.ordering[0].rootIndex] != 0):
        return 1
    for d in deltas:
        hs = rootsys.cartan_matrix @ d.hs
        f = False   
        for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    f=True
                    break
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    f=True
                    break
        if(f):
            break
    s = 0
    for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    s+=1
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    s+=weights[v]
    return s

In [ ]:
def periodicity_conj_root(rootsys:rootSystem, weights) :
    return periodicity_conj(rootsys,rootsys.SL(rootsys.delta),weights)

In [45]:
def check_periodicity_conj_rootsystem(rootsys:rootSystem):
    if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
        for l in rootsys.baseWeights[:-1]:
            if(l[rootsys.ordering[0].rootIndex] == 0):
                period = rootsys.get_periodicity(l)
                conjperiod = periodicity_conj_root(rootsys,l)
                if(period != conjperiod):
                    yield (str(rootsys.ordering),l,period,conjperiod)

In [46]:
def check_periodicity_conj_perms(perms):
    for p in perms:
        yield check_periodicity_conj_rootsystem(p)

In [47]:
def check_basic_periodicity_perms(perms,total,k=1):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_basic_periodicity,k=k),perms),total=total):
            if(result is not None):
                yield result

### Critical Roots

In [48]:
def check_lca_critical_roots_perms(perms,total=0,k=5):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(lca_on_critical_roots,k=k),perms),total=total):
            if result is not None:
                yield result

### Convexity

In [49]:
def check_convexity_perms(perms,total=0,k=5,word_convexity=False):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(convexity_from_perm,k=k,word_convexity=word_convexity),perms),total=total):
            if(result is not None):
                yield result

In [ ]:
def check_min_right_factor_gamma_im(rootsys:rootSystem,n=5):
    for delta_val in range(n):
        for base_weight in rootsys.baseWeights[:-1]:
            if(delta_val == 0 and sum(base_weight) == 1):
                continue
            w = rootsys.SL(base_weight + rootsys.delta * delta_val)[0]
            rightstand = rootsys.standfac(w)[1]
            min_right = rootsys.get_min_right_factor(w.weights)
            if sum(rightstand.weights - min_right.weights) == 0:
                continue
            if(rootsys.is_imaginary_height(sum(rightstand.weights - min_right.weights))):
                print(f"{str(rootsys.ordering)}: {base_weight}: {delta_val}: {rightstand.weights}: {min_right.weights}")

In [ ]:
def check_equiv_of_ell_on_standard_fac(rootsys:rootSystem):
    rootsys.generate_up_to_delta(1)
    for w in rootsys.baseWeights[:-1]:
        ell = rootsys.max_im_word(w)
        standw = rootsys.standfac(ell)[0].weights        
        standhs = (standw - (rootsys.delta *standw[0]))
        standhs = -standhs[1:]
        if(not rootsys.split_e_bracket(standhs,rootsys.SL(w)[0])):
            print(f"Exception: {rootsys.ordering}, {ell.no_commas()}")

In [52]:
def check_intersection_im(rootsys:rootSystem,alpha):
    for d in rootsys.get_decompositions(alpha):
        if(rootsys.is_imaginary_height(sum(d[0]))
           or rootsys.is_imaginary_height(sum(d[1]))):
            continue
        s1 = rootsys.get_imaginary_convex_set(d[0])
        s2 = rootsys.get_imaginary_convex_set(d[1])
        flag = False
        for s in s1:
            for p in s2:
                if s ==p:
                    flag = True
                    break
            if(flag):
                break
        if(not flag):
            print(d[0])
            print(d[1])
            print(*[i.no_commas() for i in s1])
            print(*[i.no_commas() for i in s2])

In [53]:
def check_chain_bound_by_m(rootsys:rootSystem,weight,k=1):
    rootsys.generate_up_to_delta(k)
    chain = rootsys.get_chain(weight)
    convexset  = rootsys.get_imaginary_convex_set(weight)
    minconvex = convexset[-1]
    maxconvex = convexset[0]
    for i in chain:
        if(maxconvex < i):
            return True
        if minconvex < i:
            return False
    return True
    

In [54]:
def check_chain_bound_rootsys(rootsys:rootSystem,genheight=3,k=1):
    rootsys.generate_up_to_delta(genheight)
    for i in rootsys.baseWeights[:-1]:
        if(not check_chain_bound_by_m(rootsys,i,k)):
            yield i

## Type A

In [46]:
n=5
Aperms = list(gen_permutations('A',n))

In [137]:
periodicity_conj_root(rootSystem([0,1,2,3,4],'A'),[0,0,1,0,0])

1

In [ ]:
for i in check_delta_type_prediction_perms(Aperms):
    print(i)

In [ ]:
write_delta_types_to_file(Aperms,"ADeltaTypes.csv")

In [ ]:
get_max_periodicity(Aperms)

0it [00:00, ?it/s]

KeyboardInterrupt: 

## Type C

In [251]:
Cperms = gen_permutations('C',6)

In [ ]:
for i in check_periodicity_conj_perms(Cperms):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Cperms):
    print(i)

In [281]:
generate_delta_types(rootSystem([0,1,2,3,4,5,6],'C'))

[1,0,2] 1


In [307]:
write_delta_types_to_file(Cperms,"CDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(Cperms)

In [ ]:
check_h_deltas_repeat(Cperms)

[]

In [ ]:
for i in generate_equivalences(Cperms):
    print(f"{i[0]} = {i[1]}")

In [ ]:
get_max_periodicity(Cperms)

(5, '0<3<1<4<2', array([0, 0, 2, 2, 1]))

## Type G

In [271]:
n=2
G2perms = gen_permutations('G',n)

In [352]:
for i in check_lca_critical_roots_perms(G2perms,math.factorial(n+1),k=10):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

('1<0<2', '120122122', '121220', '122')
('0<1<2', '01222012221', '012212', '01222')
('0<2<1', '01222012212', '01222', '012221')
('1<2<0', '1222122210', '122102', '1222')
('2<1<0', '2210221021', '221210', '2210')
('2<0<1', '2210221021', '221210', '2210')


In [166]:
check_standard_fac_same_parrallel(G2perms,k=3)

  0%|          | 0/5 [00:00<?, ?it/s]

[]

In [ ]:
for i in check_periodicity_conj_perms(G2perms):
    print(i)

In [ ]:
get_max_periodicity(G2perms)

(5, '1<0<2', array([1, 1, 3]))

In [ ]:
for i in check_delta_type_prediction_perms(G2perms):
    print(i)

In [272]:
write_delta_types_to_file(G2perms,"GDeltaTypes.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [ ]:
check_monotone_increasing_imp_cond(G2perms)

[]

In [226]:
for i in check_convexity_perms(G2perms,math.factorial(n+1),k=10,word_convexity=True):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
check_h_deltas_repeat(G2perms)

[]

In [ ]:
for i in generate_equivalences(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [273]:
Bperms = gen_permutations('B',6)

In [ ]:
for i in check_delta_type_prediction_perms(Bperms):
    print(i)

In [274]:
write_delta_types_to_file(Bperms,"BDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Bperms)

[]

In [ ]:
for i in generate_equivalences(Bperms):
    print(f"{i[0]} = {i[1]}")

## Type D

In [275]:
Dperms = gen_permutations('D',6)

In [ ]:
for i in check_delta_type_prediction_perms(Dperms):
    print(i)

In [276]:
write_delta_types_to_file(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Dperms)

[]

In [ ]:
for i in generate_equivalences(Dperms):
    print(f"{i[0]} = {i[1]}")

## Type E

In [180]:
Eperms = gen_permutations('E',6)

In [150]:
E6 = rootSystem([4,2,1,5,3,6,0],'E')
E6.generate_up_to_delta(4)
standard_delta_pattern(E6,6)

True

In [181]:
write_delta_types_to_file(Eperms,'E6DeltaTypes.csv')

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
for i in check_periodicity_conj_perms(Eperms):
    print(i)

In [ ]:
for i in check_one_smallest_last(Eperms):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Eperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
check_h_deltas_repeat(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [ ]:
for i in Eperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [28]:
write_delta_types_to_file(Eperms,"E7DeltaTypes.csv")

  0%|          | 0/40319 [00:00<?, ?it/s]

In [ ]:
for i in generate_equivalences(Eperms):
    print(f"{i[0]} = {i[1]}")

0<1<2<4<5<6<3 = 0<1<2<4<5<3<6
0<1<2<5<4<6<3 = 0<1<2<5<4<3<6
0<1<3<4<5<6<2 = 0<1<3<4<5<2<6
0<1<3<5<4<6<2 = 0<1<3<5<4<2<6
0<1<4<2<5<6<3 = 0<1<4<2<5<3<6
0<1<4<3<5<6<2 = 0<1<4<3<5<2<6
0<1<4<5<2<6<3 = 0<1<4<5<2<3<6
0<1<4<5<3<6<2 = 0<1<4<5<3<2<6
0<1<4<5<6<2<3 = 0<1<4<5<2<3<6
0<1<4<5<6<2<3 = 0<1<4<5<2<6<3
0<1<4<5<6<3<2 = 0<1<4<5<3<2<6
0<1<4<5<6<3<2 = 0<1<4<5<3<6<2
0<1<5<2<4<6<3 = 0<1<5<2<4<3<6
0<1<5<3<4<6<2 = 0<1<5<3<4<2<6
0<1<5<4<2<6<3 = 0<1<5<4<2<3<6
0<1<5<4<3<6<2 = 0<1<5<4<3<2<6
0<1<5<4<6<2<3 = 0<1<5<4<2<3<6
0<1<5<4<6<2<3 = 0<1<5<4<2<6<3
0<1<5<4<6<3<2 = 0<1<5<4<3<2<6
0<1<5<4<6<3<2 = 0<1<5<4<3<6<2
0<2<1<4<5<6<3 = 0<2<1<4<5<3<6
0<2<1<5<4<6<3 = 0<2<1<5<4<3<6
0<2<3<4<5<1<6 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<5<1<6
0<2<3<4<6<1<5 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<6<1<5
0<2<3<5<4<6<1 = 0<2<3<5<4<1<6
0<2<3<5<6<1<4 = 0<2<3<5<1<6<4
0<2<3<6<4<5<1 = 0<2<3<6<4<1<5
0<2<4<1<5<6<3 = 0<2<4<1<5<3<6
0<2<4<3<5<1<6 = 0<2<4<3<1<5<6
0<2<4<3<5<

KeyboardInterrupt: 

In [ ]:
get_max_periodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [182]:
n=4
Fperms = gen_permutations('F',4)

In [168]:
check_standard_fac_same_parrallel(Fperms,k=3)

  0%|          | 0/119 [00:00<?, ?it/s]

[]

In [ ]:
for i in check_periodicity_conj_perms(Fperms):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Fperms):
    print(i)

In [183]:
write_delta_types_to_file(Fperms,"FDeltaTypes.csv")

  0%|          | 0/119 [00:00<?, ?it/s]

In [113]:
for i in check_convexity_perms(Fperms,math.factorial(n+1),k=5,word_convexity=True):
    print(i)

  0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
check_h_deltas_repeat(Fperms)

[]

In [114]:
get_max_periodicity(Fperms,math.factorial(5))

  0%|          | 0/120 [00:00<?, ?it/s]

(9, '0<2<3<1<4', array([0, 2, 3, 4, 2]))

In [ ]:
for i in generate_equivalences(Fperms):
    print(f"{i[0]} = {i[1]}")